In [ ]:
from torchvision import datasets
from torch.utils.data import DataLoader
from torchvision.transforms import ToTensor, ToPILImage
from torch import nn
import torch

In [ ]:
train_dataset = datasets.FashionMNIST(root='data', train=True, transform=ToTensor(), download=True)
test_dataset = datasets.FashionMNIST(root='data', train=False, transform=ToTensor(), download=True)


In [3]:
labels = ('T-shirt/top',
'Trouser',
'Pullover',
'Dress',
'Coat',
'Sandal',
'Shirt',
'Sneaker',
'Bag',
'Ankle boot');

In [4]:
for batch, (X, y) in [(x, v) for x, v in enumerate(train_dataset) if x < 20]:
    print(f'sample:  {labels[y]}')
    ToPILImage()(X).show()

sample:  Ankle boot


sample:  T-shirt/top


sample:  T-shirt/top


sample:  Dress


sample:  T-shirt/top


...

sample:  Ankle boot


sample:  Trouser


sample:  T-shirt/top


sample:  Shirt


sample:  Coat


In [5]:
len(train_dataset)

60000

In [6]:
# set([x[1] for x in train_dataset]) # list(range(10)) categories

In [43]:
class NossonBasic(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.layers = nn.Sequential(
            nn.Linear(28*28, 1026), nn.ReLU(),
            nn.Linear(1026, 256), nn.ReLU(),
            nn.Linear(256, 1026), nn.ReLU(),
            nn.Linear(1026, 10)
        )
    def forward(self, x):
        x = self.flatten(x)
        logits = self.layers(x)
        return logits


In [44]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cuda device


In [45]:
n = NossonBasic().to(device)

In [46]:
n.forward(train_dataset[0][0].to(device)) # random output logits for given model params at any point (currently randomly initialized)

tensor([[ 0.0010,  0.0640,  0.0090,  0.0259,  0.0580, -0.0094,  0.0461, -0.0317,
         -0.0074, -0.0409]], device='cuda:0', grad_fn=<AddmmBackward0>)

In [47]:
n(train_dataset[0][0].to(device))

tensor([[ 0.0010,  0.0640,  0.0090,  0.0259,  0.0580, -0.0094,  0.0461, -0.0317,
         -0.0074, -0.0409]], device='cuda:0', grad_fn=<AddmmBackward0>)

In [48]:
loss_fn = nn.CrossEntropyLoss()

In [49]:
optimizer = torch.optim.Adam(n.parameters(), lr=1e-3)

#### Inspecting params / grads

In [50]:
n.layers[0].weight[0].shape

torch.Size([784])

In [51]:
list(n.layers[0].parameters())[1].grad # unset

In [52]:
n.layers[0].bias.grad

In [53]:
n.layers[0].bias, n.layers[0].bias.shape, n.layers[0].weight.shape

(Parameter containing:
 tensor([ 0.0230, -0.0035,  0.0314,  ...,  0.0036, -0.0079,  0.0331],
        device='cuda:0', requires_grad=True),
 torch.Size([1026]),
 torch.Size([1026, 784]))

In [54]:
n.layers[1], n.layers[2].weight.shape

(ReLU(), torch.Size([256, 1026]))

In [55]:
# loss for a single item:
l = loss_fn(n.forward(train_dataset[0][0].to(device)), torch.tensor(train_dataset[0][1]).unsqueeze(dim=0).to(device)); l

tensor(2.3556, device='cuda:0', grad_fn=<NllLossBackward0>)

In [56]:
l.backward() # calculate grads through computational graph
n.layers[0].bias.grad

tensor([ 0.0000,  0.0011,  0.0000,  ...,  0.0000, -0.0009,  0.0000],
       device='cuda:0')

In [57]:
pre_update_bias = n.layers[0].bias.clone();pre_update_bias # unchanged - for comparison below

optimizer.step() # use Adam optimizer to determine param update step

new_bias = n.layers[0].bias # updated bias param for layer 0

pre_update_bias, new_bias

(tensor([ 0.0230, -0.0035,  0.0314,  ...,  0.0036, -0.0079,  0.0331],
        device='cuda:0', grad_fn=<CloneBackward0>),
 Parameter containing:
 tensor([ 0.0230, -0.0045,  0.0314,  ...,  0.0036, -0.0069,  0.0331],
        device='cuda:0', requires_grad=True))

In [58]:
optimizer.zero_grad() # reset grads
n.layers[0].bias.grad

tensor([0., 0., 0.,  ..., 0., 0., 0.], device='cuda:0')

### Training

In [59]:
def train(model, dataloader, loss_fn, optimizer):
    model.train()
    num_batches = len(dataloader)
    loss = 0
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)
        logits = model(X)
        loss = loss_fn(logits, y)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        
    print(f'training loss: {loss}')

In [60]:
def test(model, dataloader, loss_fn):
    model.eval()
    size = len(dataloader.dataset)
    num_batches = len(dataloader)

    loss, correct = 0, 0
    with torch.no_grad():
        for batch, (X, y) in enumerate(dataloader):
            X, y = X.to(device), y.to(device)
            preds = model(X)
            loss += loss_fn(preds, y)
            correct += (preds.argmax(dim=1) == y).type(torch.float).sum().item()
    correct /= size
    loss /= num_batches
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {loss:>8f} \n")
    return (correct, loss)


In [64]:
train_dataloader = DataLoader(train_dataset, batch_size=100)
test_dataloader = DataLoader(test_dataset, batch_size=100)

In [65]:
num_epochs = 10

In [66]:
for _ in range(num_epochs):
    train(n, train_dataloader, loss_fn, optimizer)
    test(n, test_dataloader, loss_fn)

training loss: 0.13741345703601837
Test Error: 
 Accuracy: 88.1%, Avg loss: 0.368958 

training loss: 0.16642636060714722
Test Error: 
 Accuracy: 88.7%, Avg loss: 0.378666 

training loss: 0.1566809117794037
Test Error: 
 Accuracy: 88.7%, Avg loss: 0.394716 

training loss: 0.14958135783672333
Test Error: 
 Accuracy: 88.7%, Avg loss: 0.373710 

training loss: 0.14078761637210846
Test Error: 
 Accuracy: 88.8%, Avg loss: 0.392758 

training loss: 0.16878116130828857
Test Error: 
 Accuracy: 88.9%, Avg loss: 0.405709 

training loss: 0.1363779604434967
Test Error: 
 Accuracy: 88.7%, Avg loss: 0.435390 

training loss: 0.14530792832374573
Test Error: 
 Accuracy: 88.9%, Avg loss: 0.425578 

training loss: 0.13109569251537323
Test Error: 
 Accuracy: 88.8%, Avg loss: 0.439227 

training loss: 0.10050459206104279
Test Error: 
 Accuracy: 89.0%, Avg loss: 0.469630 



In [67]:
torch.save(n.state_dict(), 'fashion_model.pth')

In [68]:
reloaded = NossonBasic().to(device)
reloaded.load_state_dict(torch.load('fashion_model.pth'))

<All keys matched successfully>

In [69]:
test(reloaded, test_dataloader, loss_fn)

Test Error: 
 Accuracy: 89.0%, Avg loss: 0.469630 



(0.89, tensor(0.4696, device='cuda:0'))

In [71]:
reloaded.eval()
x, y = test_dataset[0][0], test_dataset[0][1]
with torch.no_grad():
    x = x.to(device)
    pred = reloaded(x)

In [72]:
pred

tensor([[-17.3578, -16.0570, -28.8215, -24.9298, -27.8089,  -9.0431, -27.0112,
         -11.2281, -22.2133,   5.6192]], device='cuda:0')

In [73]:
pred.shape

torch.Size([1, 10])

In [76]:
pred[0].argmax(0)

tensor(9, device='cuda:0')